## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [8]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

print(tf.__version__)

1.13.1


In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [10]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=32,
                             kernel_size=(3,3),
                             activation='relu',
                             input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=16,
                             kernel_size=(3,3),
                             activation='relu',))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(1,1)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 16)        4624      
_________________________________________________________________
batch_normalization_6 (Batch (None, 13, 13, 16)        64        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2704)              0         
__________

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [12]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=50)

Epoch 1/50
50000/50000 [==============================] - 7s 137us/step - loss: 1.3270 - acc: 0.5419
Epoch 2/50
50000/50000 [==============================] - 6s 123us/step - loss: 0.8905 - acc: 0.6863
Epoch 3/50
50000/50000 [==============================] - 6s 123us/step - loss: 0.7190 - acc: 0.7488
Epoch 4/50
50000/50000 [==============================] - 6s 123us/step - loss: 0.5896 - acc: 0.7954
Epoch 5/50
50000/50000 [==============================] - 6s 123us/step - loss: 0.4758 - acc: 0.8338
Epoch 6/50
50000/50000 [==============================] - ETA: 0s - loss: 0.3704 - acc: 0.871 - 6s 123us/step - loss: 0.3707 - acc: 0.8714
Epoch 7/50
50000/50000 [==============================] - 6s 123us/step - loss: 0.2912 - acc: 0.8999
Epoch 8/50
50000/50000 [==============================] - 6s 123us/step - loss: 0.2191 - acc: 0.9248
Epoch 9/50
50000/50000 [==============================] - 6s 123us/step - loss: 0.1741 - acc: 0.9402
Epoch 10/50
50000/50000 [============================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [13]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.3365559e-05, 6.2410893e-18, 5.3491163e-01, 4.6505851e-01,
        1.4855715e-05, 1.9581171e-13, 4.4467964e-08, 1.3159723e-13,
        1.5214645e-06, 1.1643458e-11]], dtype=float32)